In [4]:
# !pip install langchain_chroma

In [5]:
# !pip install chromadb

LoadDocument -- > Splitting into chunks\n
embedding --> embed chunks\n
vector chunks --> save chromadb\n
query --> similarity search chromadb\n

In [17]:
from langchain_chroma import Chroma

In [16]:
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_community.document_loaders.text import TextLoader
from langchain_ollama.embeddings import OllamaEmbeddings

In [10]:
loader = TextLoader("some_data//FDR_State_of_Union_1944.txt")
data = loader.load()
print(len(data))

1


In [13]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500)
docs = splitter.split_documents(data)

In [14]:
print(len(docs))

15


In [19]:
from langchain_ollama.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model = "llama3.2:1b")

In [21]:
db = Chroma.from_documents(documents = docs, embedding=embeddings, persist_directory="./chromadb")

In [23]:
db._persist_directory

'./chromadb'

In [24]:
db_new = Chroma(embedding_function=embeddings, persist_directory='./chromadb')

In [25]:
new_text = "What did FDR says about the cost of food law?"
similar_docs = db_new.similarity_search(new_text)

In [33]:
loader = TextLoader("some_data//Lincoln_State_of_Union_1862.txt")
data = loader.load()

In [34]:
docs = splitter.split_documents(data)
new_db = Chroma.from_documents(docs, embedding=embeddings, persist_directory='./chromadb')

Created a chunk of size 611, which is longer than the specified 500
Created a chunk of size 539, which is longer than the specified 500
Created a chunk of size 686, which is longer than the specified 500


In [40]:
docs = new_db.similarity_search("What did FDR says about the cost of food law?")
print(docs[0].page_content)
print(docs[0].metadata)

I believe the country will agree that those statements are the solemn truth.

National service is the most democratic way to wage a war. Like selective service for the armed forces, it rests on the obligation of each citizen to serve his Nation to his utmost where he is best qualified.

It does not mean reduction in wages. It does not mean loss of retirement and seniority rights and benefits. It does not mean that any substantial numbers of war workers will be disturbed in their present jobs. Let these facts be wholly clear.

Experience in other democratic Nations at warâ€”Britain, Canada, Australia, and New Zealand- has shown that the very existence of national service makes unnecessary the widespread use of compulsory power. National service has proven to be a unifying moral force based on an equal and comprehensive legal obligation of all people in a Nation at war.

There are millions of American men and women who are not in this war at all. It is not because they do not want to be 

### Here i can see that i can see the data is added to the existing database successfully and be able to fetch the data from both the documents

In [41]:
type(new_db)

langchain_chroma.vectorstores.Chroma

In [42]:
retriever = new_db.as_retriever()

In [43]:
results = retriever.get_relevant_documents("Cost food of law")

C:\Users\spurusho\AppData\Local\Temp\ipykernel_11736\3407284085.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents("Cost food of law")


In [44]:
results

[Document(id='275d6835-31fb-47b4-8894-cfb1aaee3622', metadata={'source': 'some_data//FDR_State_of_Union_1944.txt'}, page_content='(2) A continuation of the law for the renegotiation of war contractsâ€”which will prevent exorbitant profits and assure fair prices to the Government. For two long years I have pleaded with the Congress to take undue profits out of war.\n\n(3) A cost of food lawâ€”which will enable the Government (a) to place a reasonable floor under the prices the farmer may expect for his production; and (b) to place a ceiling on the prices a consumer will have to pay for the food he buys. This should apply to necessities only; and will require public funds to carry out. It will cost in appropriations about one percent of the present annual cost of the war.\n\n(4) Early reenactment of. the stabilization statute of October, 1942. This expires June 30, 1944, and if it is not extended well in advance, the country might just as well expect price chaos by summer.\n\n(5) A natio